# Momeentum signal at 14h - Live data

### Import Library

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import time
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
import requests

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120

import warnings
warnings.filterwarnings('ignore')

In [2]:
USER_AGENTS = [
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36"
]

HEADERS = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
VNDIRECT_DATA_HISTORY_URL = 'https://dchart-api.vndirect.com.vn/dchart/history'
URL = VNDIRECT_DATA_HISTORY_URL

In [3]:
def getStockHistoryData(ticker, timestamp_from=0, timestamp_to=0):
    if timestamp_from == 0:
        three_months = date.today() + relativedelta(months=-3)
        timestamp_from = datetime.strptime(three_months.strftime("%m/%d/%Y") + ', 00:00:0', "%m/%d/%Y, %H:%M:%S")\
            .timestamp()
    if timestamp_to == 0:
        timestamp_to = datetime.strptime(date.today().strftime("%m/%d/%Y") + ', 23:59:00', "%m/%d/%Y, %H:%M:%S")\
            .timestamp()

    params = {
        "resolution": "5",
        "symbol": str(ticker),
        "from": int(timestamp_from),
        "to": int(timestamp_to)
    }

    x = requests.get(URL, params=params, headers=HEADERS)
    response = x.json()

    import numpy as np
    import pandas as pd

    timestamp = np.array(response['t']).astype(int)
    close = np.array(response['c']).astype(float)
    open = np.array(response['o']).astype(float)
    high = np.array(response['h']).astype(float)
    low = np.array(response['l']).astype(float)
    volume = np.array(response['v']).astype(int)

    dataset = pd.DataFrame({'Time': timestamp, 'Open': list(open), 'High': list(high), 'Low': list(low),
                            'Close': list(close), 'Volume': list(volume)},
                           columns=['Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
    return dataset

In [4]:
def prepareData(htd):
    if 'Time' in htd.columns:
        from datetime import datetime

        htd['DateStr'] = htd.apply(
            lambda x: datetime.fromtimestamp(x['Time']).strftime("%Y-%m-%d %H:%M:%S"), axis=1)

    htd['Date'] = pd.to_datetime(htd['DateStr'])
    ticker_data = htd.set_index('Date')
    ticker_data.drop(columns=['Time', 'DateStr'], inplace=True)
    return ticker_data

### Load Price Data

In [5]:
ticker = "VN30F1M"
htd = getStockHistoryData(ticker, 0, 0)
ticker_data = prepareData(htd)
ticker_data = ticker_data.dropna()
ticker_data

,Open,High,Low,Close,Volume
Date,,,,,
2024-09-04 09:00:00,1320.8,1321.5,1317.3,1318.0,13091
2024-09-04 09:05:00,1317.8,1317.9,1310.5,1312.0,9730
2024-09-04 09:10:00,1312.0,1313.0,1311.7,1312.3,6335
2024-09-04 09:15:00,1312.1,1313.7,1312.0,1313.7,5060
2024-09-04 09:20:00,1313.6,1313.8,1311.3,1312.3,5987
...,...,...,...,...,...
2024-12-04 14:15:00,1306.8,1306.8,1304.3,1304.9,7251
2024-12-04 14:20:00,1305.6,1306.0,1304.1,1304.9,6792
2024-12-04 14:25:00,1305.0,1305.7,1303.4,1305.3,7387


In [6]:
def prepare_data(htd):
    data_tmp = htd.copy()
    data_tmp['first_close'] = data_tmp.Close
    data_tmp['Close_price'] = data_tmp.Close
    data_tmp['_last'] = data_tmp.Close
    data_tmp['prev_high'] = data_tmp.High
    data_tmp['prev_low'] = data_tmp.Low
    day_data = data_tmp.resample("D").agg({
        'first_close': cal_first_close,
        'prev_high': cal_high_before,
        'prev_low': cal_low_before,
        '_last': cal_last
    }).rename(columns={'_last': 'day_Close'})
    day_data.dropna(subset=['first_close'], inplace=True)
    day_data['prev_day_Close'] = day_data['day_Close'].shift(1)
    day_data.drop(columns=['day_Close'], inplace=True)
    day_data = day_data.dropna()

    data = htd.assign(time_d=pd.PeriodIndex(htd.index, freq='1D').to_timestamp())
    data = pd.merge(data, day_data, left_on="time_d", right_index=True, how="left")
    data.dropna(inplace=True)

    data["current"] = data.index + pd.DateOffset(minutes=5)
    data['mom_y'] = 100 * (data.Close - data.prev_day_Close) / data.prev_day_Close
    data['body_rate'] = (data.Close - data.first_close) / (data.prev_high - data.prev_low)
    data['signal'] = data.apply(lambda r: cal_signal(r), axis=1)
    data['signal_shift'] = data['signal'].shift(1)
    data.dropna(inplace=True)
    return data


def cal_signal(row):
    signal = ''
    if row['Close'] - row['prev_low'] > 21 or row['prev_high'] - row['Close'] > 21:
        return signal
    if row['mom_y'] > 0.26 and row['body_rate'] > .65:
        signal = 'long'
    elif row['mom_y'] < -.18 and row['body_rate'] < -.39:
        signal = 'short'
    return signal


def cal_first_close(tick):
    tick = tick[100 * tick.index.hour + tick.index.minute == 915]
    if not tick.empty:
        return tick[0]


def cal_high_before(tick):
    tick = tick[100 * tick.index.hour + tick.index.minute < 1345]
    return tick.max()


def cal_low_before(tick):
    tick = tick[100 * tick.index.hour + tick.index.minute < 1355]
    return tick.min()


def cal_last(tick):
    tick = tick[100 * tick.index.hour + tick.index.minute == 1445]
    if not tick.empty:
        return tick[0]

In [7]:
data = prepare_data(ticker_data)

In [8]:
data

,Open,High,Low,Close,Volume,time_d,first_close,prev_high,prev_low,prev_day_Close,current,mom_y,body_rate,signal,signal_shift
Date,,,,,,,,,,,,,,,
2024-09-05 09:05:00,1315.4,1315.5,1314.9,1315.3,2224,2024-09-05,1315.7,1318.6,1312.8,1314.0,2024-09-05 09:10:00,0.098935,-0.068966,,
2024-09-05 09:10:00,1315.3,1315.7,1315.3,1315.5,1517,2024-09-05,1315.7,1318.6,1312.8,1314.0,2024-09-05 09:15:00,0.114155,-0.034483,,
2024-09-05 09:15:00,1315.5,1316.0,1315.3,1315.7,2351,2024-09-05,1315.7,1318.6,1312.8,1314.0,2024-09-05 09:20:00,0.129376,0.000000,,
2024-09-05 09:20:00,1315.5,1316.0,1315.0,1315.7,3073,2024-09-05,1315.7,1318.6,1312.8,1314.0,2024-09-05 09:25:00,0.129376,0.000000,,
2024-09-05 09:25:00,1315.6,1315.7,1315.1,1315.4,1684,2024-09-05,1315.7,1318.6,1312.8,1314.0,2024-09-05 09:30:00,0.106545,-0.051724,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-04 14:15:00,1306.8,1306.8,1304.3,1304.9,7251,2024-12-04,1308.3,1312.8,1302.8,1314.0,2024-12-04 14:20:00,-0.692542,-0.340000,,
2024-12-04 14:20:00,1305.6,1306.0,1304.1,1304.9,6792,2024-12-04,1308.3,1312.8,1302.8,1314.0,2024-12-04 14:25:00,-0.692542,-0.340000,,
2024-12-04 14:25:00,1305.0,1305.7,1303.4,1305.3,7387,2024-12-04,1308.3,1312.8,1302.8,1314.0,2024-12-04 14:30:00,-0.662100,-0.300000,,


In [10]:
data.tail(10)

,Open,High,Low,Close,Volume,time_d,first_close,prev_high,prev_low,prev_day_Close,current,mom_y,body_rate,signal,signal_shift
Date,,,,,,,,,,,,,,,
2024-12-04 13:50:00,1309.2,1309.5,1307.8,1307.8,3546,2024-12-04,1308.3,1312.8,1302.8,1314.0,2024-12-04 13:55:00,-0.471842,-0.05,,
2024-12-04 13:55:00,1307.7,1310.2,1307.5,1308.6,6670,2024-12-04,1308.3,1312.8,1302.8,1314.0,2024-12-04 14:00:00,-0.410959,0.03,,
2024-12-04 14:00:00,1308.8,1309.9,1305.5,1305.8,7726,2024-12-04,1308.3,1312.8,1302.8,1314.0,2024-12-04 14:05:00,-0.624049,-0.25,,
2024-12-04 14:05:00,1305.8,1306.4,1305.1,1306.4,5849,2024-12-04,1308.3,1312.8,1302.8,1314.0,2024-12-04 14:10:00,-0.578387,-0.19,,
2024-12-04 14:10:00,1306.3,1307.7,1305.5,1306.7,5413,2024-12-04,1308.3,1312.8,1302.8,1314.0,2024-12-04 14:15:00,-0.555556,-0.16,,
2024-12-04 14:15:00,1306.8,1306.8,1304.3,1304.9,7251,2024-12-04,1308.3,1312.8,1302.8,1314.0,2024-12-04 14:20:00,-0.692542,-0.34,,
2024-12-04 14:20:00,1305.6,1306.0,1304.1,1304.9,6792,2024-12-04,1308.3,1312.8,1302.8,1314.0,2024-12-04 14:25:00,-0.692542,-0.34,,
2024-12-04 14:25:00,1305.0,1305.7,1303.4,1305.3,7387,2024-12-04,1308.3,1312.8,1302.8,1314.0,2024-12-04 14:30:00,-0.662100,-0.30,,
2024-12-04 14:30:00,1304.5,1304.5,1304.5,1304.5,60,2024-12-04,1308.3,1312.8,1302.8,1314.0,2024-12-04 14:35:00,-0.722983,-0.38,,
